In [1]:
!git clone https://github.com/ultralytics/yolov5.git
%cd Custom_YoloV5/yolov5

[WinError 3] The system cannot find the path specified: 'Custom_YoloV5/yolov5'
c:\Users\91808\Documents\PW\7. Deep Learning\4. Object Detection


Cloning into 'yolov5'...
C:\Users\91808\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [39]:
!pip install -r requirements.txt

In [2]:
%cd Custom_YoloV5/yolov5
!curl -L "https://app.roboflow.com/ds/87xlDYwZRa?key=9AGS5pPZrx" > SKU.zip; unzip SKU.zip; rm SKU.zip

[WinError 3] The system cannot find the path specified: 'Custom_YoloV5/yolov5'
c:\Users\91808\Documents\PW\7. Deep Learning\4. Object Detection\yolov5


C:\Users\91808\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   910  100   910    0     0    652      0  0:00:01  0:00:01 --:--:--   653
100   910  100   910    0     0    652      0  0:00:01  0:00:01 --:--:--   653

  0 1005k    0  5118    0     0   2108      0  0:08:08  0:00:02  0:08:06  2108
  6 1005k    6 68819    0     0  22609      0  0:00:45  0:00:03  0:00:42  100k
 43 1005k   43  432k    0     0   107k      0  0:00:09  0:00:04  0:00:05  264k
 97 1005k   97  980k  

In [1]:
import yaml
with open('SKU/data.yaml', 'r') as stream:
    num_classes = yaml.safe_load(stream)['nc']
print(num_classes)

3


In [2]:
with open('models/yolov5s.yaml', 'r') as stream:
    data = yaml.safe_load(stream)  
data

{'nc': 80,
 'depth_multiple': 0.33,
 'width_multiple': 0.5,
 'anchors': [[10, 13, 16, 30, 33, 23],
  [30, 61, 62, 45, 59, 119],
  [116, 90, 156, 198, 373, 326]],
 'backbone': [[-1, 1, 'Conv', [64, 6, 2, 2]],
  [-1, 1, 'Conv', [128, 3, 2]],
  [-1, 3, 'C3', [128]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [-1, 6, 'C3', [256]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [-1, 9, 'C3', [512]],
  [-1, 1, 'Conv', [1024, 3, 2]],
  [-1, 3, 'C3', [1024]],
  [-1, 1, 'SPPF', [1024, 5]]],
 'head': [[-1, 1, 'Conv', [512, 1, 1]],
  [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 6], 1, 'Concat', [1]],
  [-1, 3, 'C3', [512, False]],
  [-1, 1, 'Conv', [256, 1, 1]],
  [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 4], 1, 'Concat', [1]],
  [-1, 3, 'C3', [256, False]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [[-1, 14], 1, 'Concat', [1]],
  [-1, 3, 'C3', [512, False]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [[-1, 10], 1, 'Concat', [1]],
  [-1, 3, 'C3', [1024, False]],
  [[17, 20, 23], 1, 'Detect', ['nc', 'anchors']]

In [24]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [25]:
num_classes

3

In [26]:
%%writetemplate models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 2  # model depth multiple
width_multiple: 1  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [3]:
with open('models/yolov5s.yaml', 'r') as stream:
    data = yaml.safe_load(stream)  
data

{'nc': 80,
 'depth_multiple': 0.33,
 'width_multiple': 0.5,
 'anchors': [[10, 13, 16, 30, 33, 23],
  [30, 61, 62, 45, 59, 119],
  [116, 90, 156, 198, 373, 326]],
 'backbone': [[-1, 1, 'Conv', [64, 6, 2, 2]],
  [-1, 1, 'Conv', [128, 3, 2]],
  [-1, 3, 'C3', [128]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [-1, 6, 'C3', [256]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [-1, 9, 'C3', [512]],
  [-1, 1, 'Conv', [1024, 3, 2]],
  [-1, 3, 'C3', [1024]],
  [-1, 1, 'SPPF', [1024, 5]]],
 'head': [[-1, 1, 'Conv', [512, 1, 1]],
  [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 6], 1, 'Concat', [1]],
  [-1, 3, 'C3', [512, False]],
  [-1, 1, 'Conv', [256, 1, 1]],
  [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 4], 1, 'Concat', [1]],
  [-1, 3, 'C3', [256, False]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [[-1, 14], 1, 'Concat', [1]],
  [-1, 3, 'C3', [512, False]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [[-1, 10], 1, 'Concat', [1]],
  [-1, 3, 'C3', [1024, False]],
  [[17, 20, 23], 1, 'Detect', ['nc', 'anchors']]

In [ ]:
import torch
%cd Custom_YoloV5/yolov5
!python train.py --img 640 --batch 5 --epochs 5 --data SKU/data.yaml --cfg models/custom_yolov5s.yaml --weights yolov5s.pt --name custom-yolov5s-results --cache

In [ ]:
from utils.plots import plot_results
Image(filename='/train/yolov5s_results/results.png', width=1000)